In [ ]:
from math import sqrt
from skimage import data
from skimage.feature import blob_dog, blob_log, blob_doh
from skimage.color import rgb2gray
from skimage import io
from skimage.morphology import erosion, square

import matplotlib.pyplot as plt
from IPython.display import display

In [ ]:
# image = data.hubble_deep_field()[0:500, 0:500]
image = io.imread('../Datasets/Dataset_BUSI_with_GT/benign/benign (36)_mask.png')
print(type(image))
# image = data.coins()[50:-50, 50:-50]
image_gray = rgb2gray(image) if len(image.shape) == 3 else image
# image_gray = image
print(image_gray.shape)

blobs_log = blob_log(image_gray, max_sigma=30, num_sigma=10, threshold=0.1)
print(blobs_log.shape)

# Compute radii in the 3rd column.
# blobs_log[:, 2] = blobs_log[:, 2] * sqrt(2)
blobs_log[:, 2] = blobs_log[:, 2]

blobs_dog = blob_dog(image_gray, max_sigma=30, threshold=0.1)
print(blobs_dog.shape)
# blobs_dog[:, 2] = blobs_dog[:, 2]

blobs_doh = blob_doh(image_gray, max_sigma=30, threshold=0.01)
print(blobs_doh.shape)
# print(blobs_doh[:,0])
# print(blobs_doh[:,1])
# print(blobs_doh[:,2])

blobs_list = [blobs_log, blobs_dog, blobs_doh]
colors = ['red', 'red', 'red']
titles = ['Laplacian of Gaussian', 'Difference of Gaussian', 'Determinant of Hessian']
sequence = zip(blobs_list, colors, titles)

fig, axes = plt.subplots(1, 5, figsize=(9, 3), sharex=True, sharey=True)
ax = axes.ravel()

for idx, (blobs, color, title) in enumerate(sequence):
    ax[idx].set_title(title)
    ax[idx].imshow(image)
    for blob in blobs:
        y, x, r = blob
        c = plt.Circle((x, y), r, color=color, linewidth=2, fill=False)
        ax[idx].add_patch(c)
    ax[idx].set_axis_off()

nidx = idx+1
ax[nidx].set_title("OG Image")
e_image = erosion(image)
ax[nidx].imshow(image)
nidx += 1
ax[nidx].set_title("Eroded Image")
ax[nidx].imshow(e_image)

plt.tight_layout()
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

from skimage import data
from skimage.filters import threshold_otsu
from skimage.segmentation import clear_border
from skimage.measure import label, regionprops
from skimage.morphology import closing, square
from skimage.color import label2rgb


image = data.coins()[50:-50, 50:-50]

# apply threshold
thresh = threshold_otsu(image)
bw = closing(image > thresh, square(3))



# remove artifacts connected to image border
cleared = clear_border(bw)

# label image regions
label_image = label(cleared)
# to make the background transparent, pass the value of `bg_label`,
# and leave `bg_color` as `None` and `kind` as `overlay`
image_label_overlay = label2rgb(label_image, image=image, bg_label=0)

fig, ax = plt.subplots(figsize=(10, 6))
ax.imshow(image_label_overlay)
# ax.imshow(bw)

for region in regionprops(label_image):
    # take regions with large enough areas
    if region.area >= 100:
        # draw rectangle around segmented coins
        minr, minc, maxr, maxc = region.bbox
        rect = mpatches.Rectangle(
            (minc, minr),
            maxc - minc,
            maxr - minr,
            fill=False,
            edgecolor='red',
            linewidth=2,
        )
        ax.add_patch(rect)

ax.set_axis_off()
plt.tight_layout()
plt.show()

In [ ]:
import numpy as np

In [ ]:
x = np.zeros((10,10))

In [ ]:
a, b, r = 4, 6, 3

In [ ]:
y = x

In [ ]:
y[(a-r+1):(a+r),(b-r+1):(b+r)] = 1

In [ ]:
len(y.shape)

In [ ]:
import torch

In [ ]:
yt = torch.tensor(y)
print(yt)

In [ ]:
import os
fnames = os.listdir('../Datasets/Dataset_BUSI_with_GT/benign/')
for fname in fnames:
#     print(fname)
    x = fname.split('_')
    if len(x) > 2:
        print(x[2].split('.')[0])